In [1]:
### "Hyperparameters
r1 = 2
r2 = 2
r3 = 2
p = 8

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np
from scipy.linalg import svd
from torch.utils.data import Dataset, DataLoader
from numpy import linalg as LA
from time import perf_counter 

### '' Define the Network Structure: Linear
### Type: LTR
class NetLinear(nn.Module):

    def __init__(self, r1, r2, r3, p, N):
        
        self.r1 = r1
        self.r2 = r2
        self.r3 = r3
        self.p = p
        self.N = N
        super(NetLinear, self).__init__()
        # .conv1: 1 input matrix channel (N*P), r2 output channels, Nx1 convolution kernel
        # .conv2: 1 input matrix channel (1*P), r3 output channels, 1xr3 convolution kernel (kernel sharing)
        self.conv1 = nn.Conv2d(1, r2, kernel_size=(N, 1), bias=False) # stride is set to be (0,1) -> only move to the right
        self.conv2 = nn.Conv2d(1, r3, kernel_size=(1, p), bias=False)   # stride is set to be 0 -> no moving needed
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(in_features=r2*r3, out_features=r1, bias=False)  # 6*6 from image dimension
        self.fc2 = nn.Linear(in_features=r1, out_features=N, bias=False)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = self.conv1(x)
        z = self.conv2(x[:, :1, :, :])
        for i in range(1, x.shape[1]):
            z = torch.cat([z, self.conv2(x[:, i:(i+1), :, :])], dim = 1) #Flattening is achieved     
        z = z.view(-1, self.r2*self.r3) #-1 helps us figure out the batchsize
        x = self.fc1(z) #activation can be added on the inside as well
        x = self.fc2(x)
        return x
    
# Type: RRR
class RRR(nn.Module):
    
    def __init__(self, p, N, r1):
        self.p = p
        self.N = N
        self.r1 = r1
        super(RRR, self).__init__()
        self.fc1 = nn.Linear(in_features = N*p, out_features = r1, bias = True)
        self.fc2 = nn.Linear(in_features = r1, out_features = N, bias = True)
    def forward(self, x):
        x = self.fc1(x)
        y = self.fc2(x)
        return y

In [3]:
### "Generate Linear Time Series Inputs and Targetted Output
# return large transition Matrix A
def kronecker(A, B):
    return torch.ger(A.view(-1), B.view(-1)).reshape(*(A.size() + B.size())).permute([0, 2, 1, 3]).reshape(A.size(0)*B.size(0),A.size(1)*B.size(1))

### " F-norm of A
class L2LossFun(nn.Module):
    
    def __init__(self):
        super(L2LossFun, self).__init__()
    def forward(self, A_Est, A_True):
        gap = math.sqrt(torch.sum((A_Est - A_True)**2))
        return gap

class genA_true:
    
    def genA(r1, r2, r3, p, N):
        
        def genU(x, y, r, trans):
            # x, y-stands for the size of the random matrix
            # r stands for the numbers of dimensions to keep
            # trans is a bool to judge whether to transpose or not
            URM = torch.randn(x, y)
            U, s, VT = svd(URM)
            if trans == True:
                Us = torch.tensor(np.transpose(U[:,:r]))
            else:
                Us = torch.tensor(U[:,:r])
            return Us

        G1 =  F.normalize(torch.randn(r1, r2*r3), p=2, dim=0)*0.9 # norm of G is fixed to be 0.95 -> l2 operator norm
        #G1 =  (F.normalize(torch.randn(r1*r2*r3), p=2, dim=0)*0.95).view(r1, r2*r3) # norm of G is fixed to be 0.95 -> Frobenius norm
        U1 = genU(N, N, r1, False)
        U2T = genU(N, N, r2, True)
        U3T = genU(p, p, r3, True)
        A_dim1 = torch.mm(torch.mm(U1, G1), kronecker(U3T,U2T))
        A_dim2 = torch.eye(N*p)[:N*(p-1),:]
        A = torch.cat((A_dim1, A_dim2), dim=0)
        return A
    
def rearrangeG(K):
    K = K.permute(1,0)
    i = j = 0
    for j in range(r3):
            for i in range(r2):
                if i == 0 and j == 0:
                    tmp = K[:1,:]
                else:
                    tmp = torch.cat([tmp, K[(r3*i+j):(r3*i+1+j),:]], dim = 0)   
    tmp = tmp.permute(1,0)
    return(tmp)

def Param_Matrix(net, p, N):
    
    U2T = torch.squeeze(net.conv1.weight).view(r2, N)
    U3T = torch.squeeze(net.conv2.weight).view(r3, p)
    G1 = rearrangeG(net.fc1.weight)
    U1 = net.fc2.weight
    A = torch.mm(torch.mm(U1, G1), kronecker(U3T,U2T))
    return A

def Param_MatrixR(net):

    A = torch.mm(net.fc2.weight,net.fc1.weight)
    return A


In [4]:
### "We can use our method to generate RandomDataset
# For our linear settings burnt in is needed
class RandomDataset(Dataset):
    
    def __init__(self, p, N, Smp_size, A):
        self.X = []
        self.y = []
        for i in range(Smp_size+500):
            if i == 0:
                input_TS = torch.randn(1, 1, N, p)*200
                self.X.append(torch.squeeze(input_TS.view(1,1,N,p), dim = 0))
                input_tmp = torch.squeeze(torch.tensor(input_TS.permute(0, 1, 3, 2)).view(1,1,N*p))
                output_TS = torch.mv(A, input_tmp)
                self.y.append(output_TS[:N] + torch.randn(N))  
            else:
                input_TS = torch.cat([self.y[i-1].view(1,1,N,1), input_TS], dim = 3)
                self.X.append(torch.squeeze(input_TS[:,:,:N,:p], dim = 0))
                input_tmp = torch.squeeze(torch.tensor(input_TS[:,:,:N,:p].permute(0, 1, 3, 2)).view(1,1,N*p))
                out_tmp = torch.mv(A, input_tmp)
                self.y.append(out_tmp[:N] + torch.randn(N))
        self.X = self.X[500:]
        self.y = self.y[500:]
                
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
    def __len__(self):
        return len(self.X)

In [5]:
# input to GPU
#device = torch.device("cuda:2")
#device2 = torch.device('cpu')

In [6]:
batch_size = 1600
Smp_size = 1600
N = 36
LSresult = True

setting = "r2P8N36_015"
# initialize dict with dynamic list for storage
names = {}
names["estError" + str(setting)] =  []
names["predError" + str(setting)] =  []
names["estErrorR" + str(setting)] =  []
names["predErrorR" + str(setting)] =  []
if(LSresult == True):
    names["estErrorLS" + str(setting)] =  []
    names["predErrorLS"+ str(setting)] =  []


In [7]:
from torch.autograd import Variable
import torch.optim as optim

specRadius = 1000

while specRadius > 1:
    A_True = genA_true.genA(r1, r2, r3, p, N)
    w, v = LA.eig(A_True)  
    specRadius = max(abs(w))

distance = L2LossFun()

t1_start = perf_counter() 

for iter in range(200):
    #A_True = A_True.to(device2)
    ds = RandomDataset(p=p, N=N, Smp_size=Smp_size, A=A_True)
    ### Generate one-step ahead forecast value
    X_t, y_t = ds[Smp_size-1]
    X_F = torch.cat([y_t.view(1,1,N,1), X_t.view(1,1,N,p)], dim = 3)[:,:,:N,:p]
    y_F = torch.mv(A_True, torch.squeeze(torch.tensor(X_F[:,:,:N,:p].permute(0, 1, 3, 2)).view(1,1,N*p)))[:N] + torch.randn(N)
    
    ds = DataLoader(ds, batch_size=batch_size, shuffle=False)
    
    ### "" LTR
    net = NetLinear(r1, r2, r3, p, N)
    #net = net.to(device)

    criterion = nn.MSELoss()
    optimizer = optim.SGD(net.parameters(), lr = 0.01, momentum=0.9)

    loss_last = 1000
    loss_new = 0
    
    i = 0
    while abs(loss_last - loss_new) > 0.000000001:
        if i > 0:
            loss_last = loss_new  
        for ix, (_x, _y) in enumerate(ds):
            #=========make inpur differentiable=======================
            _x = Variable(_x).float()
            #_x = _x.to(device)
            _y = torch.squeeze(Variable(_y).float())
            #_y = _y.to(device)
            #========forward pass=====================================
            yhat = net(_x).float()
            loss = criterion(yhat, _y)
            #=======backward pass=====================================
            optimizer.zero_grad() # zero the gradients on each pass before the update
            loss.backward() # backpropagate the loss through the model
            optimizer.step() # update the gradients w.r.t the loss
            A_Est = Param_Matrix(net, p, N)
            loss_new = loss.item()
            #A_True = A_True.to(device)
        i = i + 1
        
    X_F = Variable(X_F).float()
    #X_F = X_F.to(device)
    y_F = torch.tensor(Variable(y_F).float())
    #y_F = y_F.to(device)
    predError = distance(net(X_F).float(), y_F)
    print("PredError is {}.".format(predError))
    names["estError" + str(setting)].append(distance(A_Est, A_True[:N,:]))
    print("Distance between Estimation and Truth is {}.".format(distance(A_Est, A_True[:N,:])))
    names["predError" + str(setting)].append(predError)
    
    ### "" RRR
    netR = RRR(p, N, r1)
    #netR = netR.to(device)

    criterion = nn.MSELoss()
    optimizerR = optim.SGD(netR.parameters(), lr = 0.01, momentum=0.9)

    loss_last = 1000
    loss_new = 0
    
    i = 0
    while abs(loss_last - loss_new) > 0.0000001:
        if i > 0:
            loss_last = loss_new  
        for ix, (_x, _y) in enumerate(ds):
            _x = torch.squeeze(_x.permute(0, 1, 3, 2).reshape(Smp_size, 1, p*N).permute(1, 0, 2))
            _x = Variable(_x).float()
            #_x = _x.to(device)
            _y = torch.squeeze(Variable(_y).float())
            #_y = _y.to(device)
            yhat = netR(_x).float()
            loss = criterion(yhat, _y)
            optimizerR.zero_grad() 
            loss.backward() 
            optimizerR.step() 
            A_EstR = Param_MatrixR(netR)
            loss_new = loss.item()
        i = i + 1
        
    X_F = Variable(torch.squeeze(X_F.permute(0, 1, 3, 2).reshape(1, 1, p*N).permute(1, 0, 2))).float()
    #X_F = X_F.to(device)
    y_F = torch.tensor(Variable(y_F).float())
    #y_F = y_F.to(device)
    predErrorR = distance(netR(X_F).float(), y_F)
    names["predErrorR" + str(setting)].append(predErrorR)
    print("PredError RRR is {}.".format(predErrorR))
    names["estErrorR" + str(setting)].append(distance(A_EstR, A_True[:N,:]))
    print("Distance between Estimation and Truth RRR is {}.".format(distance(A_EstR, A_True[:N,:])))
    
    ### "" LS method
    if(LSresult == True):
        for ix, (_x, _y) in enumerate(ds):
            X = _x.permute(0, 1, 3, 2).reshape(Smp_size, 1, p*N)
            X = torch.squeeze(X.permute(1, 0, 2))
            X_T = torch.transpose(X, 0, 1)
            XTX = torch.mm(X_T, X)
            XTX_Inv = torch.inverse(XTX)
            Y = _y
            A_EstLS = torch.mm(XTX_Inv, torch.mm(X_T, Y))
            Yhat = torch.mm(X, A_EstLS)
            lossLS = criterion(Y, Yhat)

        #A_True = A_True.to(device2)
        X_F = Variable(X_F).float()
        #X_F = X_F.to(device2)
        y_F = torch.tensor(Variable(y_F).float())
        #y_F = y_F.to(device2)
        y_predLS = torch.squeeze(torch.mm(torch.tensor(X_F).view(1,N*p), A_EstLS))
        predErrorLS = distance(y_predLS, y_F)
        names["predErrorLS"+ str(setting)].append(predErrorLS)
        print("PredError for LS is {}.".format(predErrorLS))
        names["estErrorLS" + str(setting)].append(distance(A_EstLS, torch.transpose(A_True[:N,:],0,1)))
        print("Distance for LS between Estimation and Truth is {}.".format(distance(A_EstLS, torch.transpose(A_True[:N,:],0,1))))
    print(iter)
    

t1_stop = perf_counter() 
print("Elapsed time during the whole program in seconds:", 
                                        t1_stop-t1_start) 
result96_v2 = names
torch.save(result96_v2, "result96_v2.py")

/Users/amy/.virtualenvs/dl4cv/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/Users/amy/.virtualenvs/dl4cv/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/Users/amy/.virtualenvs/dl4cv/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/Users/amy/.virtualenvs/dl4cv/lib/python3.6/site-packages/ipykernel_launcher.py:59: UserWarning: To copy construct from a tensor, it is 

PredError is 6.725267380855139.
Distance between Estimation and Truth is 0.31354498190832764.


KeyboardInterrupt: 

In [17]:
from statistics import mean,median

print(mean(names["estError" + str(setting)]))
print(median(names["predError"+ str(setting)]))
print(mean(names["estErrorR" + str(setting)]))
print(median(names["predErrorR"+ str(setting)]))
print(mean(names["estErrorLS" + str(setting)]))
print(median(names["predErrorLS"+ str(setting)]))

0.5882854439033539
5.945706495598872
1.0886304881964168
6.041586828169727
4.065880135837282
7.1075877001207655


In [18]:
### "Hyperparameters
r1 = 2
r2 = 2
r3 = 2
p = 5
batch_size = 178
Smp_size = 178
N = 36
LSresult = False

setting = "r2P5N36_045"
# initialize dict with dynamic list for storage
names = {}
names["estError" + str(setting)] =  []
names["predError" + str(setting)] =  []
names["estErrorR" + str(setting)] =  []
names["predErrorR" + str(setting)] =  []
if(LSresult == True):
    names["estErrorLS" + str(setting)] =  []
    names["predErrorLS"+ str(setting)] =  []



In [19]:
from torch.autograd import Variable
import torch.optim as optim

specRadius = 1000

while specRadius > 1:
    A_True = genA_true.genA(r1, r2, r3, p, N)
    w, v = LA.eig(A_True)  
    specRadius = max(abs(w))

distance = L2LossFun()

t1_start = perf_counter() 

for iter in range(200):
    #A_True = A_True.to(device2)
    ds = RandomDataset(p=p, N=N, Smp_size=Smp_size, A=A_True)
    ### Generate one-step ahead forecast value
    X_t, y_t = ds[Smp_size-1]
    X_F = torch.cat([y_t.view(1,1,N,1), X_t.view(1,1,N,p)], dim = 3)[:,:,:N,:p]
    y_F = torch.mv(A_True, torch.squeeze(torch.tensor(X_F[:,:,:N,:p].permute(0, 1, 3, 2)).view(1,1,N*p)))[:N] + torch.randn(N)
    
    ds = DataLoader(ds, batch_size=batch_size, shuffle=False)
    
    ### "" LTR
    net = NetLinear(r1, r2, r3, p, N)
    #net = net.to(device)

    criterion = nn.MSELoss()
    optimizer = optim.SGD(net.parameters(), lr = 0.01, momentum=0.9)

    loss_last = 1000
    loss_new = 0
    
    i = 0
    while abs(loss_last - loss_new) > 0.000000001:
        if i > 0:
            loss_last = loss_new  
        for ix, (_x, _y) in enumerate(ds):
            #=========make inpur differentiable=======================
            _x = Variable(_x).float()
            #_x = _x.to(device)
            _y = torch.squeeze(Variable(_y).float())
            #_y = _y.to(device)
            #========forward pass=====================================
            yhat = net(_x).float()
            loss = criterion(yhat, _y)
            #=======backward pass=====================================
            optimizer.zero_grad() # zero the gradients on each pass before the update
            loss.backward() # backpropagate the loss through the model
            optimizer.step() # update the gradients w.r.t the loss
            A_Est = Param_Matrix(net, p, N)
            loss_new = loss.item()
            #A_True = A_True.to(device)
        i = i + 1
        
    X_F = Variable(X_F).float()
    #X_F = X_F.to(device)
    y_F = torch.tensor(Variable(y_F).float())
    #y_F = y_F.to(device)
    predError = distance(net(X_F).float(), y_F)
    print("PredError is {}.".format(predError))
    names["estError" + str(setting)].append(distance(A_Est, A_True[:N,:]))
    print("Distance between Estimation and Truth is {}.".format(distance(A_Est, A_True[:N,:])))
    names["predError" + str(setting)].append(predError)
    
    ### "" RRR
    netR = RRR(p, N, r1)
    #netR = netR.to(device)

    criterion = nn.MSELoss()
    optimizerR = optim.SGD(netR.parameters(), lr = 0.01, momentum=0.9)

    loss_last = 1000
    loss_new = 0
    
    i = 0
    while abs(loss_last - loss_new) > 0.0000001:
        if i > 0:
            loss_last = loss_new  
        for ix, (_x, _y) in enumerate(ds):
            _x = torch.squeeze(_x.permute(0, 1, 3, 2).reshape(Smp_size, 1, p*N).permute(1, 0, 2))
            _x = Variable(_x).float()
            #_x = _x.to(device)
            _y = torch.squeeze(Variable(_y).float())
            #_y = _y.to(device)
            yhat = netR(_x).float()
            loss = criterion(yhat, _y)
            optimizerR.zero_grad() 
            loss.backward() 
            optimizerR.step() 
            A_EstR = Param_MatrixR(netR)
            loss_new = loss.item()
        i = i + 1
        
    X_F = Variable(torch.squeeze(X_F.permute(0, 1, 3, 2).reshape(1, 1, p*N).permute(1, 0, 2))).float()
    #X_F = X_F.to(device)
    y_F = torch.tensor(Variable(y_F).float())
    #y_F = y_F.to(device)
    predErrorR = distance(netR(X_F).float(), y_F)
    names["predErrorR" + str(setting)].append(predErrorR)
    print("PredError RRR is {}.".format(predErrorR))
    names["estErrorR" + str(setting)].append(distance(A_EstR, A_True[:N,:]))
    print("Distance between Estimation and Truth RRR is {}.".format(distance(A_EstR, A_True[:N,:])))
    
    ### "" LS method
    if(LSresult == True):
        for ix, (_x, _y) in enumerate(ds):
            X = _x.permute(0, 1, 3, 2).reshape(Smp_size, 1, p*N)
            X = torch.squeeze(X.permute(1, 0, 2))
            X_T = torch.transpose(X, 0, 1)
            XTX = torch.mm(X_T, X)
            XTX_Inv = torch.inverse(XTX)
            Y = _y
            A_EstLS = torch.mm(XTX_Inv, torch.mm(X_T, Y))
            Yhat = torch.mm(X, A_EstLS)
            lossLS = criterion(Y, Yhat)

        #A_True = A_True.to(device2)
        X_F = Variable(X_F).float()
        #X_F = X_F.to(device2)
        y_F = torch.tensor(Variable(y_F).float())
        #y_F = y_F.to(device2)
        y_predLS = torch.squeeze(torch.mm(torch.tensor(X_F).view(1,N*p), A_EstLS))
        predErrorLS = distance(y_predLS, y_F)
        names["predErrorLS"+ str(setting)].append(predErrorLS)
        print("PredError for LS is {}.".format(predErrorLS))
        names["estErrorLS" + str(setting)].append(distance(A_EstLS, torch.transpose(A_True[:N,:],0,1)))
        print("Distance for LS between Estimation and Truth is {}.".format(distance(A_EstLS, torch.transpose(A_True[:N,:],0,1))))
    print(iter)
    

t1_stop = perf_counter() 
print("Elapsed time during the whole program in seconds:", 
                                        t1_stop-t1_start) 
result84 = names
torch.save(result84, "result84.py")

/Users/amy/.virtualenvs/dl4cv/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/Users/amy/.virtualenvs/dl4cv/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/Users/amy/.virtualenvs/dl4cv/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/Users/amy/.virtualenvs/dl4cv/lib/python3.6/site-packages/ipykernel_launcher.py:59: UserWarning: To copy construct from a tensor, it is 

PredError is 6.507565423824898.
Distance between Estimation and Truth is 1.5276197101241413.


/Users/amy/.virtualenvs/dl4cv/lib/python3.6/site-packages/ipykernel_launcher.py:98: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


PredError RRR is 15.45453793938602.
Distance between Estimation and Truth RRR is 8.67974155797185.
0
PredError is 6.6536504836293275.
Distance between Estimation and Truth is 1.5074804859574809.
PredError RRR is 7.27283171231967.
Distance between Estimation and Truth RRR is 5.915198918639196.
1
PredError is 5.096742277680202.
Distance between Estimation and Truth is 1.5977033314378097.
PredError RRR is 7.861711704539474.
Distance between Estimation and Truth RRR is 5.6563032790567345.
2
PredError is 6.497810581911298.
Distance between Estimation and Truth is 1.4491605104326983.
PredError RRR is 7.807430972705086.
Distance between Estimation and Truth RRR is 7.828140106491163.
3
PredError is 6.195621285554313.
Distance between Estimation and Truth is 1.3412739763489943.
PredError RRR is 7.227552455420134.
Distance between Estimation and Truth RRR is 6.274779065866738.
4
PredError is 3.8534229420161177.
Distance between Estimation and Truth is 1.9925268146045951.
PredError RRR is 10.5481

PredError RRR is 8.373608373915456.
Distance between Estimation and Truth RRR is 7.020971219513322.
42
PredError is 5.113344476249188.
Distance between Estimation and Truth is 1.5885549659380749.
PredError RRR is 7.940133206189381.
Distance between Estimation and Truth RRR is 6.837719075879363.
43
PredError is 6.965868709197676.
Distance between Estimation and Truth is 1.6178998622108371.
PredError RRR is 9.8486118480968.
Distance between Estimation and Truth RRR is 6.016593874898716.
44
PredError is 7.19587816299172.
Distance between Estimation and Truth is 1.3919022941225676.
PredError RRR is 7.762555688237494.
Distance between Estimation and Truth RRR is 5.936269210941784.
45
PredError is 4.0529543897357545.
Distance between Estimation and Truth is 1.349334070416263.
PredError RRR is 7.247848191412519.
Distance between Estimation and Truth RRR is 6.528156624841122.
46
PredError is 5.106260137652976.
Distance between Estimation and Truth is 2.05086367486889.
PredError RRR is 5.957862

PredError RRR is 9.527806995323568.
Distance between Estimation and Truth RRR is 6.225377217702197.
84
PredError is 5.397300942846467.
Distance between Estimation and Truth is 1.5980084690876957.
PredError RRR is 6.56062740675119.
Distance between Estimation and Truth RRR is 6.715475446581039.
85
PredError is 5.7701166949710805.
Distance between Estimation and Truth is 2.2025408680617433.
PredError RRR is 8.084232345873684.
Distance between Estimation and Truth RRR is 6.3606945421544765.
86
PredError is 5.353656888014487.
Distance between Estimation and Truth is 1.6417785948471526.
PredError RRR is 7.702392543301937.
Distance between Estimation and Truth RRR is 4.783309592654409.
87
PredError is 7.302516806484212.
Distance between Estimation and Truth is 1.6669747385766631.
PredError RRR is 12.479733937679658.
Distance between Estimation and Truth RRR is 6.4949003902370634.
88
PredError is 7.066397342707902.
Distance between Estimation and Truth is 1.726966983141602.
PredError RRR is 1

PredError RRR is 6.010462539823949.
Distance between Estimation and Truth RRR is 5.538247891935176.
126
PredError is 6.830924501194229.
Distance between Estimation and Truth is 1.7988462167631536.
PredError RRR is 7.15822950412943.
Distance between Estimation and Truth RRR is 6.1118530555157164.
127
PredError is 5.852376354138421.
Distance between Estimation and Truth is 1.2983463627198601.
PredError RRR is 8.672276613025673.
Distance between Estimation and Truth RRR is 6.3890689363771385.
128
PredError is 6.919018020194017.
Distance between Estimation and Truth is 1.3681817935118676.
PredError RRR is 12.362150051922603.
Distance between Estimation and Truth RRR is 6.509305308788899.
129
PredError is 6.4028257569091975.
Distance between Estimation and Truth is 1.610846356126954.
PredError RRR is 11.555041407697065.
Distance between Estimation and Truth RRR is 7.780084361817805.
130
PredError is 5.793322027010287.
Distance between Estimation and Truth is 1.2626335677696598.
PredError RR

PredError RRR is 6.81359126598752.
Distance between Estimation and Truth RRR is 7.845025069109782.
168
PredError is 6.387539069547069.
Distance between Estimation and Truth is 1.4136235098406356.
PredError RRR is 8.035890656004094.
Distance between Estimation and Truth RRR is 8.323377333119783.
169
PredError is 6.494419636863461.
Distance between Estimation and Truth is 1.5565785930531482.
PredError RRR is 7.761692210259105.
Distance between Estimation and Truth RRR is 5.795251665287166.
170
PredError is 5.460572673961249.
Distance between Estimation and Truth is 1.7418009923701054.
PredError RRR is 17.487270412069567.
Distance between Estimation and Truth RRR is 12.210340801359786.
171
PredError is 4.6403847577623605.
Distance between Estimation and Truth is 1.5690616921136171.
PredError RRR is 5.455549777654388.
Distance between Estimation and Truth RRR is 5.799356345931814.
172
PredError is 6.994341061512916.
Distance between Estimation and Truth is 1.5386552610756943.
PredError RRR

In [20]:
from statistics import mean,median

print(mean(names["estError" + str(setting)]))
print(median(names["predError"+ str(setting)]))  
print(mean(names["estErrorR" + str(setting)]))
print(median(names["predErrorR"+ str(setting)]))
if(LSresult == True):
    print(mean(names["estErrorLS" + str(setting)]))
    print(median(names["predErrorLS"+ str(setting)]))

1.5487495527363726
6.183569769765381
6.8672214905704525
7.936051155529786


In [6]:
### "Hyperparameters
r1 = 2
r2 = 2
r3 = 2
p = 8
batch_size = 576
Smp_size = 576
N = 36
LSresult = True

setting = "r2P8N36_025"
# initialize dict with dynamic list for storage
names = {}
names["estError" + str(setting)] =  []
names["predError" + str(setting)] =  []
names["estErrorR" + str(setting)] =  []
names["predErrorR" + str(setting)] =  []
if(LSresult == True):
    names["estErrorLS" + str(setting)] =  []
    names["predErrorLS"+ str(setting)] =  []


In [7]:
from torch.autograd import Variable
import torch.optim as optim

specRadius = 1000

while specRadius > 1:
    A_True = genA_true.genA(r1, r2, r3, p, N)
    w, v = LA.eig(A_True)  
    specRadius = max(abs(w))

distance = L2LossFun()

t1_start = perf_counter() 

for iter in range(37):
    #A_True = A_True.to(device2)
    ds = RandomDataset(p=p, N=N, Smp_size=Smp_size, A=A_True)
    ### Generate one-step ahead forecast value
    X_t, y_t = ds[Smp_size-1]
    X_F = torch.cat([y_t.view(1,1,N,1), X_t.view(1,1,N,p)], dim = 3)[:,:,:N,:p]
    y_F = torch.mv(A_True, torch.squeeze(torch.tensor(X_F[:,:,:N,:p].permute(0, 1, 3, 2)).view(1,1,N*p)))[:N] + torch.randn(N)
    
    ds = DataLoader(ds, batch_size=batch_size, shuffle=False)
    
    ### "" LTR
    net = NetLinear(r1, r2, r3, p, N)
    #net = net.to(device)

    criterion = nn.MSELoss()
    optimizer = optim.SGD(net.parameters(), lr = 0.01, momentum=0.9)

    loss_last = 1000
    loss_new = 0
    
    i = 0
    while abs(loss_last - loss_new) > 0.000000001:
        if i > 0:
            loss_last = loss_new  
        for ix, (_x, _y) in enumerate(ds):
            #=========make inpur differentiable=======================
            _x = Variable(_x).float()
            #_x = _x.to(device)
            _y = torch.squeeze(Variable(_y).float())
            #_y = _y.to(device)
            #========forward pass=====================================
            yhat = net(_x).float()
            loss = criterion(yhat, _y)
            #=======backward pass=====================================
            optimizer.zero_grad() # zero the gradients on each pass before the update
            loss.backward() # backpropagate the loss through the model
            optimizer.step() # update the gradients w.r.t the loss
            A_Est = Param_Matrix(net, p, N)
            loss_new = loss.item()
            #A_True = A_True.to(device)
        i = i + 1
        
    X_F = Variable(X_F).float()
    #X_F = X_F.to(device)
    y_F = torch.tensor(Variable(y_F).float())
    #y_F = y_F.to(device)
    predError = distance(net(X_F).float(), y_F)
    print("PredError is {}.".format(predError))
    names["estError" + str(setting)].append(distance(A_Est, A_True[:N,:]))
    print("Distance between Estimation and Truth is {}.".format(distance(A_Est, A_True[:N,:])))
    names["predError" + str(setting)].append(predError)
    
    ### "" RRR
    netR = RRR(p, N, r1)
    #netR = netR.to(device)

    criterion = nn.MSELoss()
    optimizerR = optim.SGD(netR.parameters(), lr = 0.01, momentum=0.9)

    loss_last = 1000
    loss_new = 0
    
    i = 0
    while abs(loss_last - loss_new) > 0.0000001:
        if i > 0:
            loss_last = loss_new  
        for ix, (_x, _y) in enumerate(ds):
            _x = torch.squeeze(_x.permute(0, 1, 3, 2).reshape(Smp_size, 1, p*N).permute(1, 0, 2))
            _x = Variable(_x).float()
            #_x = _x.to(device)
            _y = torch.squeeze(Variable(_y).float())
            #_y = _y.to(device)
            yhat = netR(_x).float()
            loss = criterion(yhat, _y)
            optimizerR.zero_grad() 
            loss.backward() 
            optimizerR.step() 
            A_EstR = Param_MatrixR(netR)
            loss_new = loss.item()
        i = i + 1
        
    X_F = Variable(torch.squeeze(X_F.permute(0, 1, 3, 2).reshape(1, 1, p*N).permute(1, 0, 2))).float()
    #X_F = X_F.to(device)
    y_F = torch.tensor(Variable(y_F).float())
    #y_F = y_F.to(device)
    predErrorR = distance(netR(X_F).float(), y_F)
    names["predErrorR" + str(setting)].append(predErrorR)
    print("PredError RRR is {}.".format(predErrorR))
    names["estErrorR" + str(setting)].append(distance(A_EstR, A_True[:N,:]))
    print("Distance between Estimation and Truth RRR is {}.".format(distance(A_EstR, A_True[:N,:])))
    
    ### "" LS method
    if(LSresult == True):
        for ix, (_x, _y) in enumerate(ds):
            X = _x.permute(0, 1, 3, 2).reshape(Smp_size, 1, p*N)
            X = torch.squeeze(X.permute(1, 0, 2))
            X_T = torch.transpose(X, 0, 1)
            XTX = torch.mm(X_T, X)
            XTX_Inv = torch.inverse(XTX)
            Y = _y
            A_EstLS = torch.mm(XTX_Inv, torch.mm(X_T, Y))
            Yhat = torch.mm(X, A_EstLS)
            lossLS = criterion(Y, Yhat)

        #A_True = A_True.to(device2)
        X_F = Variable(X_F).float()
        #X_F = X_F.to(device2)
        y_F = torch.tensor(Variable(y_F).float())
        #y_F = y_F.to(device2)
        y_predLS = torch.squeeze(torch.mm(torch.tensor(X_F).view(1,N*p), A_EstLS))
        predErrorLS = distance(y_predLS, y_F)
        names["predErrorLS"+ str(setting)].append(predErrorLS)
        print("PredError for LS is {}.".format(predErrorLS))
        names["estErrorLS" + str(setting)].append(distance(A_EstLS, torch.transpose(A_True[:N,:],0,1)))
        print("Distance for LS between Estimation and Truth is {}.".format(distance(A_EstLS, torch.transpose(A_True[:N,:],0,1))))
    print(iter)
    

t1_stop = perf_counter() 
print("Elapsed time during the whole program in seconds:", 
                                        t1_stop-t1_start) 
result97 = names
torch.save(result97, "result97.py")

/Users/amy/.virtualenvs/dl4cv/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/Users/amy/.virtualenvs/dl4cv/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/Users/amy/.virtualenvs/dl4cv/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


KeyboardInterrupt: 

In [1]:
from statistics import mean,median

print(mean(names["estError" + str(setting)]))
print(median(names["predError"+ str(setting)]))
print(mean(names["estErrorR" + str(setting)]))
print(median(names["predErrorR"+ str(setting)]))
if(LSresult == True):
    print(mean(names["estErrorLS" + str(setting)]))
    print(median(names["predErrorLS"+ str(setting)]))

NameError: name 'names' is not defined